In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langchain.llms import OpenAI
from langdetect import detect_langs
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

In [17]:
llm = OpenAI(temperature=0.2,
             openai_api_key=INSERT_KEY_HERE,
             model = "gpt-3.5-turbo-instruct"
             )

In [18]:
neutral_tweets = pd.read_csv("split_data/neutral_tweets.csv")
biden_tweets_more = pd.read_csv("split_data/biden_tweets_more.csv")
biden_tweets_less = pd.read_csv("split_data/biden_tweets_less.csv")
trump_tweets_more = pd.read_csv("split_data/trump_tweets_more.csv")
trump_tweets_less = pd.read_csv("split_data/trump_tweets_less.csv")

In [19]:
def sampling_and_naming(sample_size, biden_tweets_more, biden_tweets_less, trump_tweets_more, trump_tweets_less, neutral_tweets):
    biden_tweets_more["agent_name"] = "Biden_A"
    biden_tweets_more = biden_tweets_more.sample(sample_size)
        
    biden_tweets_less["agent_name"] = "Biden_B"
    biden_tweets_less = biden_tweets_less.sample(sample_size)
        
    trump_tweets_more["agent_name"] = "Trump_A"
    trump_tweets_more = trump_tweets_more.sample(sample_size)
    
    trump_tweets_less["agent_name"] = "Trump_B"
    trump_tweets_less = trump_tweets_less.sample(sample_size)
    
    neutral_tweets["agent_name"] = "Neutral"
    neutral_tweets = neutral_tweets.sample(sample_size)
    df = pd.concat([biden_tweets_more, biden_tweets_less, trump_tweets_more, trump_tweets_less, neutral_tweets], ignore_index=True)
    return df 

df= sampling_and_naming(50, biden_tweets_more, biden_tweets_less, trump_tweets_more, trump_tweets_less, neutral_tweets)
    

In [12]:
df["agent_name"].value_counts()

Biden_A    50
Biden_B    50
Trump_A    50
Trump_B    50
Neutral    50
Name: agent_name, dtype: int64

In [20]:
def mock_persona(tweet_examples, task):
    def get_authors_tone_description(tweet_examples):
        template = f"""
            You are an AI Bot skilled in extracting political stances from writing examples.
            Please analyze the examples provided and determine the political stance conveyed in the writing.

            % START OF EXAMPLES
            {tweet_examples}
            % END OF EXAMPLES

            Describe the political stance of the example above.
        """
        final_prompt = template.format(tweet_examples=tweet_examples)
        return llm.predict(final_prompt)
    
    template = \
        """
        % INSTRUCTIONS
        - You are an AI Bot that is very good at mimicking an author writing style.
        - Your goal is to write content with the political stance tone that is described below.
        - You're a smart AI. You can determine from the examples below whether the user is a supporter of Trump, Biden, or neither.
        - Do not go outside the tone instructions below

        % Description of the authors political stance tone:
        {authors_tone_description}

        % START OF EXAMPLES
        {tweet_examples}
        % END OF EXAMPLES

        % YOUR TASK
        1. Answering the question "{task}" using author writing style and mock author political persona
    """
    
    authors_tone_description = get_authors_tone_description(tweet_examples)
    prompt_template = \
        template.format(authors_tone_description=authors_tone_description,
                        tweet_examples=tweet_examples, task=task)
    return prompt_template

In [24]:
def main(user_letter, task, df=df):
    user_tweets = df[df['agent_name'] == user_letter]['cleaned_tweets'].to_list()
    prompt_template = mock_persona(user_tweets, task)

    tools = load_tools(['llm-math'], llm=llm)
    agent = initialize_agent(tools, llm,
                             agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                             handle_parsing_errors=True,
                             verbose=False)

    return agent.run(prompt_template)